### This database is for music app called Sparkify, the data is modeled on the following three queries:

#### First query: Selecting artist name , song title and song's length for a certain session ID and item in sessions number.
#### Second query: Selecting artist name , song title (sorted by item in sessions number) and user's first and second name for a certain user ID and Session ID.
#### Third query: Selecting every user's name that listened to a certain song.



## Part I. ETL Pipeline for Pre-Processing the Files


#### Import Python packages 

In [14]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [15]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [16]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [17]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part 2 Apache Cassandra database

#### Creating a Cluster

In [18]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [19]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [20]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## First Query

### Selecting artist name , song title and song's length for a certain session ID and item in sessions number.

In [21]:
## This query should create a table named music_library_by_session.
## with a session_id and item_in_session as the composite primary key to match the where clause of the query.

query = "CREATE TABLE IF NOT EXISTS music_library_by_session "
query = query + "(session_id int,item_in_session int, artist text,song text, length float , PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_library_by_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Doing a SELECT to verify that the data have been inserted into each table

In [23]:
query = "select artist,song,length from music_library_by_session where session_id = 338 and item_in_session = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


##### The expected output should be 'Faithless' as the artist and the song was 'Music Matters (Mark Knight Dub)' with a length of 495.307312 seconds.

### Second Query


### Selecting artist name , song title (sorted by item in sessions number) and user's first and second name for a certain user ID and Session ID.

In [24]:
## This query should create a table named music_library_by_user.
## with a user_id and session_id as the composite primary key to match the where clause of the query. 
## item_in_session as the clustering column to sort the table.
try:
    session.execute("""

        CREATE TABLE IF NOT EXISTS music_library_by_user (
        user_id int,
        session_id int,
        item_in_session int,
        artist text,
        song text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id,session_id),item_in_session))
        
        """)
except Exception as e:
    print(e)
                    

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_library_by_user (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [26]:
rows = session.execute("SELECT artist,song,first_name,last_name FROM music_library_by_user where user_id = 10 and session_id = 182")

for row in rows:
    print(row.artist,row.song,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


##### The expected output should be that Sylvie Cruz listened to 4 songs: 'Keep on Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier', and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.

## Third Query

### Selecting every user's name that listened to a certain song.

In [27]:
## This query should create a table named user_info_by_song.
## with song as the partition key to match the where clause. 
## user_id as the clustering column to allow for dublicates of the partions key with differend users.
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_info_by_song(
        song text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song,user_id))
    """)
except Exception as e:
    print(e)

                    

In [28]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO user_info_by_song (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [29]:
rows = session.execute("SELECT first_name,last_name FROM user_info_by_song where song='All Hands Against His Own'")

for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


##### The expected output should be three users listened to the song 'All Hands Against His Own': Jacqueline Lynch, Sara Johnson, and Tegan Levine.

### Droping the tables before closing out the sessions

In [30]:
session.execute("drop table music_library_by_session")
session.execute("drop table music_library_by_user")
session.execute("drop table user_info_by_song")

### Closing the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()